# Building Regularization

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/building_regularization.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [1]:
# %pip install geoai-py

## Import package

In [2]:
import geoai

ModuleNotFoundError: No module named 'geoai'

## Download sample data

In [ ]:
naip_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train.tif"
)
masks_url = "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_building_masks.tif"

In [ ]:
masks_path = geoai.download_file(masks_url)
naip_path = geoai.download_file(naip_url)

In [ ]:
geoai.view_image(masks_path, figsize=(18, 10))

In [ ]:
geoai.create_split_map(
    left_layer=masks_url,
    right_layer=naip_url,
    left_label="Building Masks",
    right_label="NAIP Imagery",
    left_args={
        "colormap": {"255": "#0000ff"},
        "opacity": 0.5,
    },
    basemap=naip_url,
)

## Convert raster to vector

In [ ]:
gdf = geoai.raster_to_vector(masks_path, output_path="naip_building_masks.geojson")

In [ ]:
geoai.view_vector_interactive(
    gdf, style_kwds={"color": "blue", "fillOpacity": 0.4}, tiles=naip_url
)

In [ ]:
geoai.create_split_map(
    left_layer=gdf,
    right_layer=naip_url,
    left_label="Buildings",
    right_label="NAIP Imagery",
    left_args={"style": {"color": "blue"}, "fillOpacity": 0.4},
    basemap=naip_url,
)

## Building regularization

In [ ]:
gdf_regularized = geoai.regularization(
    building_polygons=gdf,
    angle_tolerance=10,
    simplify_tolerance=0.5,
    orthogonalize=True,
    preserve_topology=True,
)

In [ ]:
geoai.view_vector_interactive(
    gdf_regularized, style_kwds={"color": "red", "fillOpacity": 0.4}, tiles=naip_url
)

In [ ]:
geoai.create_split_map(
    left_layer=gdf_regularized,
    right_layer=naip_url,
    left_label="Regularized Buildings",
    right_label="NAIP Imagery",
    left_args={"style": {"color": "red"}, "fillOpacity": 0.4},
    basemap=naip_url,
)

## Hybrid regularization

In [ ]:
gdf_hybrid = geoai.hybrid_regularization(gdf)

In [ ]:
geoai.view_vector_interactive(
    gdf_hybrid, style_kwds={"color": "green", "fillOpacity": 0.4}, tiles=naip_url
)

In [ ]:
geoai.create_split_map(
    left_layer=gdf_regularized,
    right_layer=naip_url,
    left_label="Regularized Buildings",
    right_label="NAIP Imagery",
    left_args={"style": {"color": "green", "fillOpacity": 0.4}},
    basemap=naip_url,
)

## Adaptive regularization

In [ ]:
gdf_adaptive = geoai.adaptive_regularization(
    building_polygons=gdf,
    simplify_tolerance=0.5,
    area_threshold=0.9,
    preserve_shape=True,
)

In [ ]:
geoai.view_vector_interactive(
    gdf_adaptive, style_kwds={"color": "yellow", "fillOpacity": 0.4}, tiles=naip_url
)

In [ ]:
geoai.create_split_map(
    left_layer=gdf_adaptive,
    right_layer=naip_url,
    left_label="Adaptive Regularization Buildings",
    right_label="NAIP Imagery",
    left_args={"style": {"color": "yellow", "fillOpacity": 0.4}},
    basemap=naip_url,
)

## Compare regularization methods

In [ ]:
import leafmap.foliumap as leafmap

In [ ]:
m = leafmap.Map()
m.add_basemap("SATELLITE")
m.add_gdf(gdf, layer_name="Original")
m.add_gdf(
    gdf_regularized, style={"color": "red", "fillOpacity": 0}, layer_name="Regularized"
)
m.add_gdf(gdf_hybrid, style={"color": "green", "fillOpacity": 0}, layer_name="Hybrid")
m.add_gdf(
    gdf_adaptive, style={"color": "yellow", "fillOpacity": 0}, layer_name="Adaptive"
)
legend = {
    "Original": "blue",
    "Regularized": "red",
    "Hybrid": "green",
    "Adaptive": "yellow",
}
m.add_legend(title="Building Footprints", legend_dict=legend)
m